<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**


*Stack Overflow est un site célèbre de question-réponses liées au développement informatique. Pour poser une question sur ce site, il faut entrer plusieurs tags de manière à retrouver facilement la question par la suite. Pour les utilisateurs expérimentés cela ne pose pas de problème, mais pour les nouveaux utilisateurs, il serait judicieux de suggérer quelques tags relatifs à la question posée.*

*Amateur de Stack Overflow, qui vous a souvent sauvé la mise, vous décidez d'aider la communauté en retour. Pour cela, vous développez un système de suggestion de tags pour le site. Celui-ci prendra la forme d’un algorithme de machine learning qui assigne automatiquement plusieurs tags pertinents à une question.*



**Solutions mises en oeuvre**

Les solutions de suggestion de tags présentés ici se basent sur des modèles supervisés de machine learning.

Les modèles mis en oeuvre : 
    * Multinomial Naive Bayes
    * Regression logistic
    * SGD
    * Multinomial Bernoulli 

Ce notebook utilise les données issues des notebooks : 


**`P6_DadaAnalysis.ipynb`**

**`P6_UnsupervizedMethods.ipynb`**



In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# <font color=blus>4. Supervized methods</font>

## <font color=blus>4.0. Encoding TAGs issued from train and test data-sets</font>

* Multiple TAGs are assigned to a unique POST.

    This lead to deal with a multi-label classification issue.

    All labels (TAGs) are one hot encoded.


* Train and test data-sets do not have same assigned TAGs set.

    This leads to a different number of encoded columns for each data-set.

    For having same number of columns for train and test datasets, encoding process uses the union of train and 
    test assigned TAGs .



In [ ]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags_test=pd.read_csv(file_name)['Tags']
print(ser_sof_tags_test.shape)

file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags_train=pd.read_csv(file_name)['Tags']
print(ser_sof_tags_train.shape)



**<font color='blue'>List of unique TAGs is built</font>**

In [ ]:
import p6_util

# String of TAGs are converted as list of TAGS.
list_list_tags  = ser_sof_tags_train.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>').tolist()
print(len(list_list_tags))
list_list_tags += ser_sof_tags_test.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>').tolist()
print(len(list_list_tags))
#list_ref_tags = list(set(list_ref_tags))
#print(len(list_ref_tags))
list_ref_tags = list()
for list_tags in list_list_tags :
    list_ref_tags += [tag for tag in list_tags]

list_ref_tags = list(set(list_ref_tags))
print(len(list_ref_tags))


**<font color='blue'>TAGs from train and test datasets are encoded and saved into CSR matrix</font>**

In [ ]:
import p5_util
import p6_util

csr_matrix_encoded_train_tags, list_ref_train_tags = p6_util.p6_encode_ser_tag_2_csrmatrix(ser_sof_tags_train, list_ref_tags)
print("csr_matrix_encoded_train_tags : {}".format(csr_matrix_encoded_train_tags.shape))

file_name="./data/csr_matrix_encoded_train_tags.dump"
p5_util.object_dump(csr_matrix_encoded_train_tags,file_name)

In [ ]:
import p5_util
import p6_util

csr_matrix_encoded_test_tags, list_ref_test_tags = p6_util.p6_encode_ser_tag_2_csrmatrix(ser_sof_tags_test, list_ref_tags)
print("csr_matrix_encoded_test_tags : {}".format(csr_matrix_encoded_test_tags.shape))

file_name="./data/csr_matrix_encoded_test_tags.dump"
p5_util.object_dump(csr_matrix_encoded_test_tags,file_name)

## <font color=blus>4.2. Test dataset processing</font>

Processing results in : 
    * POSTs standardization
    * POSTs vectorization (words embedded on dictionary issued from most weighted tokens).
    * TAGs from POSTs are encoded in a one hot way and saved into CSR matrix.

In [ ]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_test=pd.read_csv(file_name)
print(df_sof_test.shape)

**<font color='blue'>Test dataset standardization</font>**


Process is applied on a POST that is built from merged `Body` and `Title` columns

In [ ]:
df_sof_test['Body']=df_sof_test['Body']+df_sof_test['Title']

In [ ]:
import p6_util
ser_sof_body_test = p6_util.p6_df_standardization(df_sof_test.Body)

**<font color='blue'>Test dataset Vectorization</font>**

* Loading TF-IDF vectorizer; this is same operator used for train data-set vectorization.

In [ ]:
import p5_util
file_name="./data/csr_matrix_tfidf_ngram1_b.dump"
csr_matrix = p5_util.object_load(file_name)
print(csr_matrix.shape)

file_name="./data/vectorizer_tfidf_ngram1_b.dump"
vectorizer = p5_util.object_load(file_name)

In [ ]:
csr_matrix_tdif_1gram_test = vectorizer.transform(ser_sof_body_test)
csr_matrix_tdif_1gram_test.shape

**<font color='blue'>Save of test CSR matrix issued from test POST vectorization</font>**

In [ ]:
import p5_util
file_name="./data/csr_matrix_tdif_1gram_test.dump"
p5_util.object_dump(csr_matrix_tdif_1gram_test,file_name)
csr_matrix_tdif_1gram_test.shape

## <font color=blus>4.3. Loading SOF TAGs</font>

Those TAGs from SOF database are used as targets for M.L. process.


In [1]:
import pandas as pd
file_name="./data/StackOverFlowTags.csv"
df_sof_tags=pd.read_csv(file_name)
df_sof_tags.shape

(50000, 2)

## <font color=blus>4.4. Loading vectorized train dataset and encoded train target dataset</font>

**Loading TF-IDF data-set containing trained vectorized POSTs**

In [2]:
import p5_util

file_name="./data/csr_matrix_tfidf_1gram.dump"
csr_matrix_tfidf_1gram = p5_util.object_load(file_name)
print(csr_matrix_tfidf_1gram.shape)

p5_util.object_load : fileName= ./data/csr_matrix_tfidf_1gram.dump
(16359, 3641)


**Loading encoded train target data-set**

In [3]:
import p5_util

file_name="./data/csr_matrix_encoded_train_tags.dump"
csr_matrix_encoded_train_tags = p5_util.object_load(file_name)
print(csr_matrix_encoded_train_tags.shape)

p5_util.object_load : fileName= ./data/csr_matrix_encoded_train_tags.dump
(24604, 8385)


## <font color=blus>4.5. Loading vectorized test dataset and encoded test target dataset</font>

**Loading TF-IDF data-set containing test vectorized POSTs**

In [4]:
import p5_util
file_name="./data/csr_matrix_tdif_1gram_test.dump"
csr_matrix_tdif_1gram_test = p5_util.object_load(file_name)
csr_matrix_tdif_1gram_test.shape

p5_util.object_load : fileName= ./data/csr_matrix_tdif_1gram_test.dump


(16359, 6304)

**Loading encoded test target data-set**

In [5]:
import p5_util

file_name="./data/csr_matrix_encoded_test_tags.dump"
csr_matrix_encoded_test_tags = p5_util.object_load(file_name)
print(csr_matrix_encoded_test_tags.shape)

p5_util.object_load : fileName= ./data/csr_matrix_encoded_test_tags.dump
(16359, 8385)


In [6]:
dict_cls_score = dict()

In [7]:
X_train = csr_matrix_tfidf_1gram
y_train = csr_matrix_encoded_train_tags

X_test = csr_matrix_tdif_1gram_test
y_test = csr_matrix_encoded_test_tags

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

col_min = min(y_train.shape[1],  y_test.shape[1] )

print("\nMin col for target : \n"+str(col_min))

if False:
    y_train = y_train[:, :col_min]
    y_test  = y_test[:,:col_min]

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(16359, 3641) (24604, 8385) (16359, 6304) (16359, 8385)

Min col for target : 
8385


## <font color='blus'>4.6. Evaluation of Multinomial Bernouilli Naive Bayes classifier</font>

**<font color='blue'>Loading SOF TAGs </font>**

In [8]:
import pandas as pd
file_name="./data/StackOverFlowTags.csv"
df_sof_tags=pd.read_csv(file_name)
df_sof_tags.shape

(50000, 2)

**<font color='blue'>Loading encoded test target data-set</font>**

In [9]:
import p5_util

file_name="./data/csr_matrix_encoded_test_tags.dump"
csr_matrix_encoded_test_tags = p5_util.object_load(file_name)
print(csr_matrix_encoded_test_tags.shape)

p5_util.object_load : fileName= ./data/csr_matrix_encoded_test_tags.dump
(16359, 8385)


**<font color='blue'>Loading TF-IDF test data-set containing vectorized POSTs</font>**

In [10]:
import p5_util
file_name="./data/csr_matrix_tdif_1gram_test.dump"
csr_matrix_tdif_1gram_test = p5_util.object_load(file_name)
csr_matrix_tdif_1gram_test.shape

p5_util.object_load : fileName= ./data/csr_matrix_tdif_1gram_test.dump


(16359, 6304)

**<font color='blue'>Loading encoded train target data-set</font>**

In [11]:
import p5_util

file_name="./data/csr_matrix_encoded_train_tags.dump"
csr_matrix_encoded_train_tags = p5_util.object_load(file_name)
print(csr_matrix_encoded_train_tags.shape)

p5_util.object_load : fileName= ./data/csr_matrix_encoded_train_tags.dump
(24604, 8385)


**<font color='blue'>Loading TF-IDF train data-set containing vectorized POSTs</font>**

In [12]:
import p5_util

file_name="./data/csr_matrix_tfidf_ngram1_b.dump"
csr_matrix_tfidf_ngram1_b = p5_util.object_load(file_name)
print(csr_matrix_tfidf_ngram1_b.shape)

p5_util.object_load : fileName= ./data/csr_matrix_tfidf_ngram1_b.dump
(24604, 6304)


In [13]:
X_train = csr_matrix_tfidf_ngram1_b
y_train = csr_matrix_encoded_train_tags

X_test = csr_matrix_tdif_1gram_test
y_test = csr_matrix_encoded_test_tags


col_min = min(y_train.shape[1],  y_test.shape[1] )

print("\nMin col for target : "+str(col_min))

if False:
    y_train = y_train[:, :col_min]
    y_test  = y_test[:,:col_min]

print("\n *** X_train= {} y_train={} X_test={} y_test={}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))



Min col for target : 8385

 *** X_train= (24604, 6304) y_train=(24604, 8385) X_test=(16359, 6304) y_test=(16359, 8385)


In [14]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.multiclass import OneVsRestClassifier
N_JOBS=-1
cls_ber = OneVsRestClassifier(BernoulliNB(), n_jobs=N_JOBS).fit(X_train, y_train)

In [ ]:
cls_ber

**Compute accuracy**

In [ ]:
y_pred = cls_ber.predict(X_test)

In [ ]:
y_pred.shape

In [ ]:
df_sof_test.columns, df_sof_test.shape[0]

In [ ]:
# A randomly POST is selected
import random
import p6_util

post_id = random.choice(range(0, y_pred.shape[0]))
for post_id in range(0, df_sof_test.shape[0]) :
    body= df_sof_test.Body.iloc[post_id]
    title= df_sof_test.Title.iloc[post_id]
    tags = df_sof_test.Tags.iloc[post_id]
    print("\n Assigned TAGs : "+tags)

    pred_row_tag = y_pred[post_id].toarray()

    tuple_arr_row_pos = np.where(pred_row_tag>0)
    print(tuple_arr_row_pos)
    for row, col in zip(tuple_arr_row_pos[0], tuple_arr_row_pos[1]):
        print("\n Predicted TAG(s) : "+df_sof_tags.TagName.tolist()[val[0]])

    break


In [ ]:
import numpy as np
for row in y_pred :


In [ ]:
# Name of predicted TAG from issued from SOF TAGs

# Get assigned TAGs for this POST


In [ ]:
import numpy as np
M = np.array([[4,1,8],[5,2,1], [1,0,10]])

tuple_index_array = np.where(M==1)
for row, col in zip(tuple_index_array[0],tuple_index_array[1]) :
    print((row, col), M[row][col])

In [ ]:
arr_index.shape, y_pred.shape

In [ ]:
from sklearn.metrics import accuracy_score

cls_ber_score = accuracy_score(y_test  ,y_pred)

In [ ]:
y_pred.shape
cls_ber.score(X_test, y_test)

In [15]:
from sklearn.metrics import accuracy_score
y_pred = cls_ber.predict(X_test)

cls_ber_score = accuracy_score(y_test[:,:col_min]  ,y_pred[:,:col_min])

print("Accuracy score for Multinomial Bernoulli classifier : {0:1.2F} %".format(cls_ber_score*100))

dict_cls_score['MN Bernoulli'] = cls_ber_score

file_name="./data/dict_cls_score.dump"
p5_util.object_dump(dict_cls_score , file_name)

Accuracy score for Multinomial Bernoulli classifier : 1.34 %


## <font color=blus>4.7. Applying Naive Bayes classifier</font>

### <font color=blus>4.7.1. Training N.B. classifier</font>

**Training classifier**

one versus Rest leads having one classifier per class.

`OneVsRestClassifier` classifier is used because of multiple classes. 

Then, one classifier per class is fitted.



In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

cls_mnb = OneVsRestClassifier(MultinomialNB(), n_jobs=-1).fit(X_train, y_train)


In [17]:
cls_mnb

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
          n_jobs=-1)

### <font color='blus'>4.7.2.Classifier evaluation</font>

**TAGs predictions and probabilities for test dataset**

In [ ]:
y_pred = cls_mnb.predict(X_test)

**<font color=blue>P({Tag_i}|Post)</font>**

In [ ]:
y_proba = model.predict_proba(X_test)

y_proba.shape

**Compute accuracy**

In [18]:
from sklearn.metrics import accuracy_score
y_pred = cls_mnb.predict(X_test)

cls_mnb_score = accuracy_score(y_test  ,y_pred)

print("Accuracy score for MNB classifier : {0:1.2F} %".format(cls_mnb_score*100))

dict_cls_score['MNB'] = cls_mnb_score

Accuracy score for MNB classifier : 0.34 %


In [19]:
import p5_util

file_name="./data/dict_cls_score.dump"
p5_util.object_dump(dict_cls_score,file_name)


## <font color='blus'>4.8. Chained Multinomial Naive Bayes classifier</font>

In [20]:
import p5_util

file_name="./data/dict_cls_score.dump"
dict_cls_score = p5_util.object_load(file_name)


p5_util.object_load : fileName= ./data/dict_cls_score.dump


In [21]:
from skmultilearn.problem_transform import ClassifierChain

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

chained_classifier_mnb = ClassifierChain(MultinomialNB())

In [22]:
chained_classifier_mnb.fit(X_train , y_train)

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: R

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: R

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: R

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: R

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: R

...

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: R

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: R

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: R

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: R

ClassifierChain(classifier=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
        order=None, require_dense=[True, True])

In [23]:
import p5_util

file_name="./data/chained_classifier_mnb.dump"
p5_util.object_dump(chained_classifier_mnb, file_name)

In [ ]:
import p5_util

file_name="./data/chained_classifier_mnb.dump"
chained_classifier_mnb = p5_util.object_load(file_name)

In [24]:
y_pred = chained_classifier_mnb.predict(X_test)

chained_classifier_mnb_score = accuracy_score(y_test  ,y_pred)

print("Accuracy mean score for Chained MNB classifier : {0:1.2F} %".format(chained_classifier_mnb_score*100))

dict_cls_score['Chained MNB'] = chained_classifier_mnb_score

Accuracy mean score for Chained MNB classifier : 0.24 %


In [25]:
import p5_util

file_name="./data/dict_cls_score.dump"
p5_util.object_dump(dict_cls_score,file_name)

## <font color='blus'>4.9. SGD classifier</font>

In [26]:
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

sgd_classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)).fit(X_train, y_train)

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 21 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 23 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 25 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 29 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packa

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 213 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 214 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 220 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 222 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 226 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 228 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 453 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 455 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 462 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 463 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 464 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 474 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 637 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 647 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 651 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 656 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 658 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 661 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 867 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 871 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 873 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 879 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 880 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 883 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

...

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7511 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7515 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7516 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7521 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7522 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7530 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7718 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7719 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7723 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7729 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7731 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7732 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7935 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7936 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7939 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7940 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7942 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7946 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8147 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8148 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8149 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8157 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8160 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8164 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8345 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8352 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8354 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8358 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8359 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8362 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

In [27]:
y_pred = sgd_classifier.predict(X_test)

sgd_classifier_score = accuracy_score(y_test[:,:col_min]  ,y_pred[:,:col_min])

print("Accuracy mean score for SGD classifier : {0:1.2F} %".format(sgd_classifier_score*100))

dict_cls_score['SGD'] = sgd_classifier_score

Accuracy mean score for SGD classifier : 1.14 %


In [28]:
import p5_util

file_name="./data/dict_cls_score.dump"
p5_util.object_dump(dict_cls_score,file_name)

## <font color='blus'>4.10. Grid Search CV Logistic regression & One vs rest</font>

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

dict_param_grid = {
    'estimator__C': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0],
    'estimator__penalty': ['l2'],
    'n_jobs': [-1]
}

estimator = OneVsRestClassifier(LogisticRegression())
gscv_lr_classifier  = GridSearchCV(estimator, dict_param_grid)

gscv_lr_classifier.fit(X_train, y_train)

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 163 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 165 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 166 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 169 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 172 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 173 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 338 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 344 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 347 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 349 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 351 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 352 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 469 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 474 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 475 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 477 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 479 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 481 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 604 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 605 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 607 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 609 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 611 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 612 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/sit

...

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6451 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6454 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6465 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6474 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6477 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6482 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6870 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6888 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6903 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6908 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6910 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6915 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7284 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7295 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7301 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7304 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7310 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7317 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7672 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7681 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7697 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7700 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7703 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7707 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8025 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8027 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8029 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8032 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8033 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8041 is present in all training examples.
  str(classes[c]))
/home/bangui/anaconda3/lib/python3

In [ ]:
print (gscv_lr_classifier.best_score_)
print (gscv_lr_classifier.best_params_)

In [ ]:
import p5_util
file_name='./data/dict_cls_score.dump'
dict_cls_score = p5_util.object_load(file_name)

dict_cls_score['GSCV Logistic Regression'] = gscv_lr_classifier.best_score_
p5_util.object_dump(dict_cls_score, file_name)

In [ ]:
dict_cls_score

## <font color='blus'>4.11. GSCV SGD classifier</font>

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
dict_param_grid = {
    'estimator__alpha': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0],
    'estimator__penalty': ['l2'],
    'n_jobs': [-1]
}

estimator = OneVsRestClassifier(SGDClassifier())
gscv_sgd_cls  = GridSearchCV(estimator, dict_param_grid, cv=5)
gscv_sgd_cls.fit(X_train, y_train)

print (gscv_sgd_cls.best_score_)
print (gscv_sgd_cls.best_params_)

In [ ]:
import p5_util
file_name='./data/dict_cls_score.dump'
dict_cls_score = p5_util.object_load(file_name)

dict_cls_score['GSCV SGD Regression'] = gscv_sgd_cls.best_score_
p5_util.object_dump(dict_cls_score, file_name)

In [ ]:
dict_cls_score

## <font color='blus'>4.12. Powered SGD classifier</font>

In [ ]:
from sklearn.linear_model import SGDClassifier
from skmultilearn.problem_transform import LabelPowerset
power_classifier_sgd = LabelPowerset(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None))
power_classifier_sgd.fit(X_train, y_train)

y_pred = power_classifier_sgd.predict(X_test)

power_classifier_sgd_score = accuracy_score(y_test[:,:col_min]  ,y_pred[:,:col_min])

print("Accuracy mean score for Powered SGD classifier : {0:1.2F} %".format(power_classifier_sgd_score*100))

dict_cls_score['Powered SGD'] = power_classifier_sgd_score

In [ ]:
import p5_util
file_name='./data/dict_cls_score.dump'
dict_cls_score = p5_util.object_load(file_name)

dict_cls_score['GSCV SGD Regression'] = gscv_sgd_cls.best_score_
p5_util.object_dump(dict_cls_score, file_name)

## <font color='blus'>4.13. Classifiers results</font>

In [ ]:
dict_cls_score

In [ ]:
import pandas as pd
df_result = pd.DataFrame.from_dict( dict_cls_score, orient='index')
df_result.reset_index(inplace=True)
df_result.rename(columns={'index':'Classifier',0:'Score'}, inplace=True)
df_result

In [ ]:
import p6_util_plot
title = "Classifiers accuracy"
p6_util_plot.ser_item_occurency_plot(df_result.Classifier, df_result.Score*100, item_count=None, title=title)